In [4]:
from dotenv import load_dotenv

load_dotenv()

True

# Part 1

In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

loader = WebBaseLoader(
    web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

retriver = vectorstore.as_retriever()

prompt = hub.pull('rlm/rag-prompt')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    model_kwargs = {
        "seed": 42
    }
    )

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke('What is Task decomposition?')

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task decomposition is a technique used to break down complex tasks into smaller, more manageable steps. This can be achieved through prompting the model to "think step by step" or by using task-specific instructions. It can also be done with human inputs.'

# Part 2 Indexing

In [6]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat"

In [7]:
import tiktoken

def num_tokens_from_string(string:str, encoding_name:str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [8]:
embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

768

In [9]:
import numpy as np 
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1*norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print(similarity)

0.8569384258805943


In [10]:
loader = WebBaseLoader(
    web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)
blog_docs = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(blog_docs)

In [12]:
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriver = vectorstore.as_retriever()

# Part 3: Retrieval

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

retriver = vectorstore.as_retriever(search_kwargs={"k":1})

In [14]:
docs = retriver.get_relevant_documents("What is task decomposition?")

C:\Users\Admin\AppData\Local\Temp\ipykernel_7548\4276928549.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriver.get_relevant_documents("What is task decomposition?")


In [15]:
len(docs)

1

# Part 4: Generation

In [16]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the folowing context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the folowing context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [17]:
chain = prompt | llm

In [19]:
chain.invoke({"context":docs, "question":"What is Task Decomposition?"})

AIMessage(content='Task decomposition is the process of breaking down a complicated task into smaller, simpler steps. It can be done by LLM with simple prompting, by using task-specific instructions, or with human inputs.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fdd599fb-8e2c-4df1-992f-2b61fa22b3e6-0', usage_metadata={'input_tokens': 325, 'output_tokens': 41, 'total_tokens': 366, 'input_token_details': {'cache_read': 0}})

In [20]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [22]:
prompt_hub_rag.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [25]:
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriver, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complicated task into smaller and simpler steps. It can be done by LLM with simple prompting, by using task-specific instructions, or with human inputs.'